In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
credit_data = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

In [ ]:
credit_data.head()

In [ ]:
round(credit_data['Attrition_Flag'].value_counts()/credit_data['Attrition_Flag'].count(),2)

16% of customers are churned out and using this we need to see how much customers will get churn from credit card services. Look like, we have bias in output and normal regression will predict most of the customers won't churn out. 

Need to normalize this bias

## **Data Defintions**
* CLIENTNUM - Client ID
* Attrition_Flag - Attrited customer if customer closed the account else Existing Customer
* Customer_Age - Age of customer
* Gender - Customer gender
* Dependent_count - Number of dependents
* Education_Level - Level of education
* Marital_Status - Marital status
* Income_Category - Income bucket
* Card_Category - Type of card (Blue,Silver,Gold,Platinum) 
* Months_on_book - Period of relationship with bank 
* Total_Relationship_Count - Total number of products held with the customer
* Months_Inactive_12_mon - Number of months inactive in last 12 months 
* Contacts_Count_12_mon - Number of contacts in last 12 months
* Credit_Limit - Credit limit on the card 
* Total_Revolving_Bal - Balance available in credit card 
* Avg_Open_To_Buy - Open to buy credit line (Average of last 12 months)
* Total_Amt_Chng_Q4_Q1 - Change in transaction amount (Q4 over Q1) 
* Total_Trans_Amt - Total transaction amount (Last 12 months) 
* Total_Trans_Ct - Total transaction count (Last 12 months) 
* Total_Ct_Chng_Q4_Q1 - Change in transaction count (Q4 over Q1) 
* Avg_Utilization_Ratio - Average card utilization ratio 

### Removing Naive bayes columns from the data

In [ ]:
a = ['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1']

credit_data = credit_data.drop(columns = a)

    
    

In [ ]:

temp = credit_data[['CLIENTNUM','Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon']]
temp.head(10)

## EDA on customers data

Formulating hypothesis: 
1. Age of customer imapcts attrition rate and usage of credit card 
2. Which gender has a good amount of education background
3. Which gender has high usage of credit card and respective dependants in the family 
4. Check whether Months on book correlates with 
5. How type of card correlates with usage of credit card 
6. Credit limit distribution over different types of credit card provided 
7. Whether Number of months inactive impacted revovling balance and utilization ratio 

In [ ]:
import seaborn as sns
import matplotlib as pyplot 
%matplotlib  inline 

from plotly import graph_objs as go

import re
import string
import numpy as np 
import random

import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

In [ ]:
ax = sns.displot( x = 'Customer_Age', data=credit_data )

In [ ]:
# Median age of credit card customer 
median_age = credit_data['Customer_Age'].median()
print(median_age)

Customer Age is normally distributed on this data. More number of customers are working professional and it implies a range between 35 to 60. Median age of customer states on 46.0 

Working professionals will get their career highest salary between 40-50. At this stage, all of them are willing to spend a lot and take a loan for thier liabilities and assests. As age and career deteriorating, they will get rid of credit card and save for retirement corpus

Let's see the how many the dependants for the customers

In [ ]:
ax = sns.catplot(x="Dependent_count", hue="Gender", col="Attrition_Flag",
                data=credit_data, kind="count",
                height=4, aspect=.7);

Eventhough existing customer and attrited customer occupies 80/20 ratio in the data. This shows that Female customers have more dependants than Men which arises two more questions here , 
1. How marital status factor impact these much dependants on female customers
2. Whether female customers use high number of credit cards 
3. Distribution of gender in our population

In [ ]:
# Distribution of Gender 

ax = sns.countplot(x = 'Gender', data = credit_data)

In [ ]:
gender_wise =  round( (credit_data['Gender'].value_counts()/len(credit_data))*100 , 0)
gender_wise

Yes. In this data, we have high number of female customers than Male but with little marginal difference. The same marginal difference we can see in the distribution of dependent in Age category 

In [ ]:
# Marital status of Female vs Male
ax = sns.countplot(x = 'Marital_Status', hue = 'Gender' , data = credit_data )

Single&Divorced women are comparatively more than Married women. This shows that financially this category women always in need of money to support their children and pay their mortages. We can see single women has their age distribution from 30 to 60. 

In [ ]:
female_marital_status = round((credit_data[credit_data['Gender']=='F']['Marital_Status'].value_counts()/len(credit_data[credit_data['Gender']=='F'])*100),0)
female_marital_status

In [ ]:
male_marital_status = round((credit_data[credit_data['Gender']=='M']['Marital_Status'].value_counts()/len(credit_data[credit_data['Gender']=='M'])*100),0)
male_marital_status

Comparing Male and Female distribution on marital status. More Female stays single than Male whereas other categories are relatively same to each other 

Any other question pop up here : Age distribution of single male and female

In [ ]:
# Age distribution 
# Age starts with 26 and goes upto 73 in our population - using np.digitize method

bins = [20, 30, 40, 50, 60 ,70]
names = ['20-30', '30-40', '40-50', '50-60', '60-70', '70-80']

d = dict(enumerate(names, 1)) 

credit_data['Age_bucket'] = np.vectorize(d.get)(np.digitize(credit_data['Customer_Age'],bins))


In [ ]:
credit_data['Age_bucket'].value_counts()

In [ ]:
age_bucket_single = credit_data[credit_data['Marital_Status']=='Single'].sort_values(by = 'Age_bucket', ascending = True)

In [ ]:
age_bucket_single.head()

In [ ]:
# Age distribution of single male and female
ax = sns.countplot(x = 'Age_bucket', hue = 'Gender' , data = age_bucket_single )

In [ ]:
single_male_age_bucket = credit_data[(credit_data['Marital_Status']=='Single') & (credit_data['Gender']=='M')]['Age_bucket'].value_counts()
single_male_age_bucket

In [ ]:
single_female_age_bucket = credit_data[(credit_data['Marital_Status']=='Single') & (credit_data['Gender']=='F')]['Age_bucket'].value_counts()
single_female_age_bucket

Single women and men are there even after 40+. That makes extra burden to the respective gender in handling the financial situation of the household. 
Questions from here for the above two categories 
1. Utilization of credit card 
2. Revovling balance in their credit card 

In [ ]:
# Utilization distribution 
# Utilization percent - using np.digitize method

bins = [0.0,0.25 ,0.50, 0.70 , 100]
names = ['0-25', '25-50', '50-75', '75-100']

d = dict(enumerate(names, 1)) 

credit_data['Utilization_bucket'] = np.vectorize(d.get)(np.digitize(credit_data['Avg_Utilization_Ratio'],bins))

In [ ]:
credit_data['Utilization_bucket'].value_counts()

In [ ]:
#Scatter plot on utilization and age bucket to find the clusters

In [ ]:
sns.barplot(data = credit_data, x=credit_data['Age_bucket'], y=credit_data['Avg_Utilization_Ratio'],estimator = np.median)

As customer ages, their need also increasing with respect to their family dependants

Correlation plot before get into data cleaning and feature extraction

In [ ]:
corr = credit_data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

See, Credit Limit is correlating with Avg_Open_To_Buy. This looks like radius correlates with circumference

Data Cleaning

In [ ]:
#checking missing values 
credit_data.describe()

This dataset doesn't have any missing values. So no need of imputation work on this. We can jump to model and tuning this bias dataset..

Starting Data preprocessing

In [ ]:
credit_data.head()

In [ ]:
credit_data.Attrition_Flag = credit_data.Attrition_Flag.replace({'Existing Customer':0, 'Attrited Customer':1})

In [ ]:
credit_data.Gender = credit_data.Gender.replace({'M':0,'F':1})
credit_data = pd.concat([credit_data, pd.get_dummies(credit_data['Education_Level']).drop(columns=['Unknown'])],axis=1)
credit_data = pd.concat([credit_data, pd.get_dummies(credit_data['Income_Category']).drop(columns=['Unknown'])],axis=1)
credit_data = pd.concat([credit_data, pd.get_dummies(credit_data['Marital_Status']).drop(columns=['Unknown'])],axis=1)
credit_data = pd.concat([credit_data, pd.get_dummies(credit_data['Card_Category']).drop(columns=['Platinum'])],axis=1)

credit_data.drop(columns = ['Education_Level','Income_Category','Marital_Status','Card_Category','CLIENTNUM'])

This dataset has 16% attrition flag and it's a high bias. On normalizing this bias, I will use "SMOTE" technique to oversampling the minority class. 

Also, undersampling the majority class in random state. On combining these both, oversampling and undersampling will provide the better overview about the dataset. This reduces the bias and variance in the dataset